Dataset Attribution:
    https://www.kaggle.com/therohk/india-headlines-news-dataset

In [1]:
import numpy as np
import pprint
pp = pprint.PrettyPrinter(indent=4)

In [2]:
categories = {}
with open("./india-news-headlines.csv", "r", encoding="utf8") as f:
    text = f.read()
    lines = text.split("\n")
    print(len(lines[1:-1]))
    for line in lines[1:-1]:
        parts = line.split(",")
        if (parts[1] not in categories):
            categories[parts[1]] = []
        categories[parts[1]].append(", ".join(parts[2:]))

2665614


Aim below (3 and 4) is to roll up sub-categories into broader categories. May be better to do this by hand eventually... should provide a better output. 

In [3]:
arr = []
final_cats = {}
for key in categories:
    arr.append((key, len(key.split("."))))
    
arr = sorted(arr, key = lambda x:x[1])

for i in range(len(arr)):
    if (arr[i][1] < 3):
        final_cats[arr[i][0]] = []

for i in range(len(arr)):
    if (arr[i][1] >= 3):
        parts = arr[i][0].split(".")
        base_phrase = ".".join(parts[0:2])
        num_found = 0
        for k in range(i+1, len(arr)):
            if (arr[k][1] >= 3 and k != i):
                if (base_phrase in arr[k][0]):
                    num_found += 1
        if (num_found > 0):
            if (base_phrase not in final_cats):
                final_cats[base_phrase] = []
            final_cats[base_phrase].append(arr[i][0])

#pp.pprint(final_cats)

In [4]:
exclude = ["city", "times", "unknown", "removed", "cities", "top-headlines", "top-stories"]
count = 0
redo = True
while (redo):
    redo = False
    del_list = {}
    for key in final_cats:
        include = True
        for i in range(len(exclude)):
            if (exclude[i] in key):
                include = False
        if (include):
            total = 0
            if (key in categories):
                total += len(categories[key])
            if (len(final_cats[key]) > 0):
                for i in range(len(final_cats[key])):
                    if (final_cats[key][i] in categories):
                        total += len(categories[final_cats[key][i]])
            if (total > 5000):
                count += 1
            else:
                parts = key.split(".")
                use_key = parts[0]
                if ("cricket" in key or "ipl" in key or "icc" in key): #this is hacky but it's the only way to keep the rest of the code clean
                    use_key = "cricket"
                if (parts[0] in final_cats and len(parts) > 1):
                    del_list[key] = use_key
                    redo = True
    
    print("Merging:", len(del_list.keys()))
    for key in del_list:
        if (del_list[key] not in final_cats):
            final_cats[del_list[key]] = []
        final_cats[del_list[key]].extend(final_cats[key])
        if (key in categories):
            final_cats[del_list[key]].append(key)
        del final_cats[key]

good_cats = []
for key in final_cats:
    include = True
    for i in range(len(exclude)):
        if (exclude[i] in key):
            include = False
    if (include):
        total = 0
        if (key in categories):
            total += len(categories[key])
        if (len(final_cats[key]) > 0):
            for i in range(len(final_cats[key])):
                if (final_cats[key][i] in categories):
                    total += len(categories[final_cats[key][i]])
        if (total > 2500): #select categories with more than 2500 datapoints
            good_cats.append(key)

Merging: 205
Merging: 0


In [5]:
#again, hacky but required
final_cats["cricket"].append("sports.icc-world-cup-2015")
final_cats["cricket"].extend(final_cats["sports.icc-world-cup-2015"])
del(good_cats[good_cats.index("sports.icc-world-cup-2015")])

In [6]:
print(good_cats, len(good_cats))

['india', 'cricket', 'news', 'entertainment', 'world', 'blogs', 'leisure', 'sports', 'tech', 'auto', 'life-style', 'nri', 'others', 'business.india-business', 'sports.tennis', 'tech.tech-news', 'world.pakistan', 'world.rest-of-world', 'world.us', 'home.science', 'business.international-business', 'sports.football', 'sports.golf', 'entertainment.events', 'life-style.health-fitness', 'home.environment', 'home.education', 'sports.more-sports', 'entertainment.kannada', 'entertainment.tamil', 'entertainment.hindi', 'entertainment.malayalam', 'life-style.relationships', 'entertainment.english', 'tv.news', 'entertainment.telugu'] 36


In [7]:
reorg = {}

for i in range(len(good_cats)):
    key = good_cats[i]
    if (key not in reorg):
        reorg[key] = []
    reorg[key].extend(categories[key])
    for sub_key in final_cats[key]:
        reorg[key].extend(categories[sub_key])
        
for key in reorg:
    print(key, len(reorg[key]))
"""
for key in final_cats:
    if (key not in reorg):
        reorg[key] = []
    for i in range(len(final_cats[key])):
        get_key = final_cats[key][i]
        reorg[key].extend(categories[get_key])
"""

india 251900
cricket 39151
news 13962
entertainment 9575
world 19394
blogs 6704
leisure 2544
sports 18823
tech 18064
auto 4035
life-style 16066
nri 3771
others 3877
business.india-business 94762
sports.tennis 7727
tech.tech-news 19069
world.pakistan 8313
world.rest-of-world 12000
world.us 15189
home.science 12146
business.international-business 28325
sports.football 22493
sports.golf 6204
entertainment.events 22295
life-style.health-fitness 13220
home.environment 7079
home.education 14272
sports.more-sports 10561
entertainment.kannada 9832
entertainment.tamil 14875
entertainment.hindi 80833
entertainment.malayalam 7984
life-style.relationships 6028
entertainment.english 30379
tv.news 28400
entertainment.telugu 6648


'\nfor key in final_cats:\n    if (key not in reorg):\n        reorg[key] = []\n    for i in range(len(final_cats[key])):\n        get_key = final_cats[key][i]\n        reorg[key].extend(categories[get_key])\n'

In [8]:
original_distribution = categories
categories = reorg

In [9]:
headlines_covered = 0
good_categories = list(categories.keys())
print(good_categories)
print(len(good_categories))
num_cat = len(good_categories)

['india', 'cricket', 'news', 'entertainment', 'world', 'blogs', 'leisure', 'sports', 'tech', 'auto', 'life-style', 'nri', 'others', 'business.india-business', 'sports.tennis', 'tech.tech-news', 'world.pakistan', 'world.rest-of-world', 'world.us', 'home.science', 'business.international-business', 'sports.football', 'sports.golf', 'entertainment.events', 'life-style.health-fitness', 'home.environment', 'home.education', 'sports.more-sports', 'entertainment.kannada', 'entertainment.tamil', 'entertainment.hindi', 'entertainment.malayalam', 'life-style.relationships', 'entertainment.english', 'tv.news', 'entertainment.telugu']
36


merge_cats = {
    "top-stories":"top-headlines"
}

for key in merge_cats:
    categories[merge_cats[key]].extend(categories[key])
    del categories[key]

In [10]:
import re
def processHeadline_One(text):
    return re.sub(r'[^\w\s]','',text)
    """
    #merge noun phrases?
    new_text = re.sub(r'[^\w\s]','',text)
    words = new_text.split(" ")
    for i in range(len(words)):
        word = words[i] 
        if (any(x.isUpper() for x in word))
    """

In [11]:
longest_phrase = 0
word_lengths = []
vocab = {}
for i in range(len(good_categories)):
    t_cat = good_categories[i]
    for k in range(len(categories[t_cat])):
        headline = processHeadline_One(categories[t_cat][k])
        words = headline.split(" ")        
        for j in range(len(words)):
            if (words[j] not in vocab):
                vocab[words[j]] = 0
            vocab[words[j]] += 1
        word_lengths.append(len(words))
print(np.mean(word_lengths), np.percentile(word_lengths, 99))

7.59103440496 15.0


In [12]:
vocab_arr = sorted([(word, vocab[word]) for word in vocab], key=lambda x:x[1], reverse=True)

In [13]:
final_vocab = {}
rev_vocab = {}
for i in range(len(vocab_arr)):
    final_vocab[vocab_arr[i][0]] = i
    rev_vocab[i] = vocab_arr[i][0]

In [14]:
print(len(vocab_arr))
print(vocab_arr[75000:75020])

180627
[('14point', 2), ('repulse', 2), ('Jyotir', 2), ('Instil', 2), ('NTS', 2), ('Panagarh', 2), ('Proving', 2), ('Contenders', 2), ('2O', 2), ('Burqas', 2), ('1880', 2), ('ModiJagan', 2), ('AntiTerrorist', 2), ('cutlery', 2), ('plantbased', 2), ('Rairakhol', 2), ('Dhanus', 2), ('postTaliban', 2), ('patronising', 2), ('Vijayabhaskar', 2)]


In [15]:
#build dataset
vocab_cutoff = 75000
alt_words = {}
alt_words["#UNKNOWN_WORD#"] = vocab_cutoff + 1
alt_words["#EOS"] = vocab_cutoff + 2
alt_words["#PADDING_WORD#"] = vocab_cutoff + 3

In [32]:
def word_index(word): #returns the lowest possible index for a word, regardless of case
    index = alt_words["#UNKNOWN_WORD#"]
    if (word in final_vocab):
        index = final_vocab[word]
    try:
        if (final_vocab[word.lower()] < index):
            index = final_vocab[word.lower()]
        if (final_vocab[word.upper()] < index):
            index = final_vocab[word.upper()]
        n_word = word[0].upper() + word[1:]
        if (final_vocab[n_word] < index):
            index = final_vocab[n_word]
    except:
        if (word in final_vocab):
            index = final_vocab[word]
    if (index > vocab_cutoff):
        index = alt_words["#UNKNOWN_WORD#"]
    return index

In [18]:
dataset = []

for i in range(len(good_categories)):
    headlines = categories[good_categories[i]]
    for k in range(len(headlines)):
        headline = processHeadline_One(headlines[k])
        indices = []
        words = headline.split(" ")
        if (len(words) <= 14):
            for x in range(len(words)):
                indices.append(word_index(words[x]))
            indices.append(alt_words["#EOS"])
            while (len(indices) < 15):
                indices.append(alt_words["#PADDING_WORD#"])
            dataset.append((indices,i))         

In [19]:
print(len(dataset))
dataset = np.array(dataset)
np.random.shuffle(dataset)

874449


In [20]:
print(dataset.shape)

(874449, 2)


In [21]:
from keras.models import Model
from keras.layers import Input, Embedding, Dense, GRU, LSTM, Dropout

Using TensorFlow backend.
/home/devadmin/.conda/envs/maria/lib/python3.6/importlib/_bootstrap.py:205: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


In [22]:
input_layer = Input((15,))
embedding = Embedding(75005, 128)(input_layer)
gru_1 = LSTM(128)(embedding)
dropout = Dropout(0.1)(gru_1)
d1 = Dense(128, activation="tanh")(dropout)
d2 = Dense(64, activation="relu")(d1)
output = Dense(num_cat, activation="sigmoid")(d2) #sigmoid used here to allow for extraction of multiple topics

model = Model(input_layer, output)

In [23]:
model.compile(loss="binary_crossentropy", optimizer="rmsprop", metrics = ["categorical_accuracy"])
print(model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 15)                0         
_________________________________________________________________
embedding_1 (Embedding)      (None, 15, 128)           9600640   
_________________________________________________________________
lstm_1 (LSTM)                (None, 128)               131584    
_________________________________________________________________
dropout_1 (Dropout)          (None, 128)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 128)               16512     
_________________________________________________________________
dense_2 (Dense)              (None, 64)                8256      
_________________________________________________________________
dense_3 (Dense)              (None, 36)                2340      
Total para

In [24]:
validation_split = 0.1
val_cutoff = int(validation_split * len(dataset))
train = dataset[val_cutoff:]
validate = dataset[:val_cutoff]

x_train = [train[i][0] for i in range(len(train))]
y_train = [train[i][1] for i in range(len(train))]

print(y_train[:10])
x_val = [validate[i][0] for i in range(len(validate))]
y_val = [validate[i][1] for i in range(len(validate))]

[30, 0, 0, 28, 30, 13, 8, 0, 23, 0]


In [25]:
print(len(x_train))

787005


In [26]:
val_acc = 0

In [27]:
step = 50000
num_full_epochs = 4
for k in range(num_full_epochs): 
    print("Restarting on full dataset...", k)
    for i in range(0, 750000, step):
        print("epoch:", k, ",", i, ":", i+step)
        x = [] 
        y = []
        for j in range(i, i+step):
            x.append(np.asarray(x_train[j]))
            arr = np.zeros(num_cat, dtype=np.int8)
            arr[y_train[j]] = 1
            y.append(arr)
        model.fit(np.asarray(x), np.asarray(y), batch_size=128, validation_split=0.1)
    print("\n\n\n\n Predicting:")
    x_ = []
    y_ = []
    for i in range(0, len(x_val)):
        x_.append(np.asarray(x_val[i]))
    output = model.predict(np.asarray(x_), verbose=True)
    indices = np.argmax(output, axis=1)
    acc = 0
    for i in range(len(indices)):
        if (indices[i] == y_val[i]):
            acc += 1
    accuracy = acc/len(indices)
    if (accuracy > val_acc):
        print("\nValidation Accuracy went up to:", accuracy, "\nSaving model.\n\n\n\n")
        model.save("./HeadlinesOfIndia.h5")
        val_acc = accuracy
    else:
        print("\nNew validation accuracy is:", accuracy, "\nNot saving.\n\n\n\n")

Restarting on full dataset... 0
epoch: 0 , 0 : 50000
Train on 45000 samples, validate on 5000 samples
Epoch 1/1
45000/45000 [==============================] - 11s - loss: 0.1119 - categorical_accuracy: 0.2939 - val_loss: 0.1025 - val_categorical_accuracy: 0.2544
epoch: 0 , 50000 : 100000
Train on 45000 samples, validate on 5000 samples
Epoch 1/1
45000/45000 [==============================] - 9s - loss: 0.0889 - categorical_accuracy: 0.3815 - val_loss: 0.0832 - val_categorical_accuracy: 0.4266
epoch: 0 , 100000 : 150000
Train on 45000 samples, validate on 5000 samples
Epoch 1/1
45000/45000 [==============================] - 10s - loss: 0.0780 - categorical_accuracy: 0.4548 - val_loss: 0.0774 - val_categorical_accuracy: 0.4430
epoch: 0 , 150000 : 200000
Train on 45000 samples, validate on 5000 samples
Epoch 1/1
45000/45000 [==============================] - 10s - loss: 0.0708 - categorical_accuracy: 0.4994 - val_loss: 0.0706 - val_categorical_accuracy: 0.5014
epoch: 0 , 200000 : 250000
T

Train on 45000 samples, validate on 5000 samples
Epoch 1/1
45000/45000 [==============================] - 10s - loss: 0.0396 - categorical_accuracy: 0.7362 - val_loss: 0.0440 - val_categorical_accuracy: 0.7038
epoch: 2 , 200000 : 250000
Train on 45000 samples, validate on 5000 samples
Epoch 1/1
45000/45000 [==============================] - 10s - loss: 0.0394 - categorical_accuracy: 0.7393 - val_loss: 0.0432 - val_categorical_accuracy: 0.7128
epoch: 2 , 250000 : 300000
Train on 45000 samples, validate on 5000 samples
Epoch 1/1
45000/45000 [==============================] - 9s - loss: 0.0388 - categorical_accuracy: 0.7407 - val_loss: 0.0423 - val_categorical_accuracy: 0.7170
epoch: 2 , 300000 : 350000
Train on 45000 samples, validate on 5000 samples
Epoch 1/1
45000/45000 [==============================] - 10s - loss: 0.0389 - categorical_accuracy: 0.7414 - val_loss: 0.0428 - val_categorical_accuracy: 0.7204
epoch: 2 , 350000 : 400000
Train on 45000 samples, validate on 5000 samples
Epoc

In [28]:
print(val_acc)

0.7319198572800878


In [29]:
for i in range(0, len(x_val)):
    x_.append(np.asarray(x_val[i]))
output = model.predict(np.asarray(x_), verbose=True)

174880/174888 [============================>.] - ETA: 0s

In [30]:
rev_vocab[75001] = "#UNKNOWN_WORD#"
rev_vocab[75002] = "#EOS"
for i in range(7000, 8000): #len(x_val)):
    sent = x_val[i]
    sent_str = ""
    for word in sent:
        if (word < 75002):
            sent_str += rev_vocab[word] + " "
    cats = output[i]
    scores = sorted([(i,cats[i]) for i in range(len(cats))], key=lambda x: x[1], reverse=True)
    potential_cats = [good_categories[scores[k][0]] for k in range(1)]
    potential_cats = "[ " + " ; ".join(potential_cats) + " ]"
    print(sent_str, potential_cats)

Taliban deny Afghan Shia kidnap  [ world ]
Sameera and Ishant Sharma  [ entertainment.hindi ]
Ali Zafar travels by local train  [ entertainment.hindi ]
Arrest #UNKNOWN_WORD# of your skin  [ life-style ]
Kashmir Valley gets a passport office  [ india ]
Vizag on a recordbreaking spree  [ entertainment.hindi ]
spotted Anushka Sharma spends quality time with Virat Kohli in Sri Lanka  [ entertainment.hindi ]
fuel cess could be used for road safety  [ india ]
Swaminathan S a Aiyar let AI fade away like #UNKNOWN_WORD#  [ blogs ]
93 Mumbai blasts Convicts seek leniency  [ india ]
Photogallery  [ entertainment ]
Ragini in a movie  [ entertainment.kannada ]
Seven goals that shock the World  [ sports.football ]
Hyderabad girls outshine boys in CBSE #UNKNOWN_WORD#  [ cricket ]
Ultrasound through #UNKNOWN_WORD# patch may become reality  [ home.science ]
Govts assurance on bhoomi pujan says VHP  [ india ]
Students of unique Goa University course graduate  [ home.education ]
Police blamed as Angola p

15 years after first test Army yet to get its Nag  [ india ]
Kirron and Kiran attend a do in Delhi  [ entertainment.events ]
Ellen Degeneres makes history at Peoples Choice Awards 2017  [ entertainment.english ]
Lydia Ko heads NZ entries for Golf at Rio Olympics  [ sports.golf ]
Police solve NRI murder mystery  [ india ]
US bond market rallies as inflation data miss forecasts  [ business.international-business ]
GSK to phase out #UNKNOWN_WORD# toothbrush  [ business.india-business ]
spot gold may rise into #UNKNOWN_WORD# range  [ business.international-business ]
Eternal heroes  [ entertainment.hindi ]
Fashions getting younger N #UNKNOWN_WORD#  [ life-style ]
Incredible India  [ entertainment.hindi ]
Afghanistan hopeful of qualifying for 2011 World Cup  [ sports.football ]
Iraq crisis Diesel decontrol plan takes a hit  [ business.international-business ]
Rajnath Singh wears skull cap to enhance #UNKNOWN_WORD#  [ india ]
Left regains Madurai after three decades  [ india ]
USL licenses e

In [33]:
test_dataset = []
test_categories = ["unknown"]
for i in range(len(test_categories)):
    headlines = original_distribution[test_categories[i]]
    for k in range(len(headlines)):
        headline = processHeadline_One(headlines[k])
        indices = []
        words = headline.split(" ")
        if (len(words) <= 14):
            for x in range(len(words)):
                indices.append(word_index(words[x]))
            indices.append(alt_words["#EOS"])
            while (len(indices) < 15):
                indices.append(alt_words["#PADDING_WORD#"])
            test_dataset.append((indices,i)) 

In [34]:
x_test = [test_dataset[i][0] for i in range(len(test_dataset))]

In [35]:
print(x_test[10])

[16615, 2051, 6499, 12, 47047, 75002, 75003, 75003, 75003, 75003, 75003, 75003, 75003, 75003, 75003]


In [37]:
x_ = []
for i in range(0, len(x_test)):
    x_.append(np.asarray(x_test[i]))
output = model.predict(np.asarray(x_), verbose=True)

220768/220950 [============================>.] - ETA: 0s

In [41]:
unknown_categorised = []

rev_vocab[75001] = "#UNKNOWN_WORD#"
rev_vocab[75002] = "#EOS"
for i in range(len(x_test)):
    sent = x_test[i]
    sent_str = ""
    for word in sent:
        if (word < 75002):
            sent_str += rev_vocab[word] + " "
    new_headline = {}
    new_headline["headline"] = sent_str

    cats = output[i]
    scores = sorted([(i,cats[i]) for i in range(len(cats))], key=lambda x: x[1], reverse=True)
    potential_cats = [good_categories[scores[k][0]] for k in range(3)]
    new_headline["categories"] = potential_cats
    unknown_categorised.append(new_headline)
    #potential_cats = "[ " + " ; ".join(potential_cats) + " ]"
    #print(sent_str, potential_cats)

In [51]:
import json
json_str = json.dumps(unknown_categorised, indent=4)
with open("./testing_category_unknown.json", "w") as f:
    json.dump(unknown_categorised, f, indent=4)